In [ ]:
# import das bibliotecas
import numpy as np
import pandas as pd
import cv2
from glob import glob 
import os
import matplotlib.pyplot as plt
import tqdm

from sklearn.metrics import classification_report, log_loss, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split

import xgboost as xgb

np.random.seed(42)

### Realizando o a leitura do dataset

Leitura do dataset, no momento estamos realizando testes com apenas 10 mil imagens.

In [ ]:
path = "/kaggle/input/histopathologic-cancer-detection/" 
labels = pd.read_csv(path + 'train_labels.csv')
train_path = path + 'train/'

In [ ]:
df = pd.DataFrame({'path': glob(os.path.join(train_path,'*.tif'))})
df['id'] = df.path.map(lambda x: ((x.split("n")[-1].split('.')[0])[1:]))
df = df.merge(labels, on = "id")
df.head(3)

In [ ]:
IMG_SIZE = 96
BATCH_SIZE = 128

In [ ]:
plt.title("Distribuição das classes");

plt.pie(df['label'].value_counts(), labels=['Sem cancer',
          'Com Cancer'], startangle=180, autopct='%1.1f', 
           colors=['#00ff99','#FF96A7'], shadow=True);
plt.figure(figsize=(16,16));
plt.show();

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X = []
y = []
for idx in tqdm.tqdm(range(df.shape[0])):

    X.append(cv2.imread(df.iloc[idx]['path']))
    y.append(df.iloc[idx]['label'])
    if idx == 10000:
        break
X = np.array(X)
y = np.array(y)

In [ ]:
plt.title("Primeira imagem do dataset")
imagem = cv2.imread(df['path'][0], cv2.IMREAD_GRAYSCALE)
plt.imshow(imagem);

In [ ]:
# Dividindo o dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Sobre o Classificador

O XGBoost  (e**X**treme **G**radient **Boost**ing) é um algoritmo de classificação (possui versão pra regressão também) baseado em árvore de decisão com Gradient Boosting (Aumento de gradiente).

### Sobre Gradient Boosting

Gradient Boosting é uma técnica que utiliza de *ensemble* de modelos considerados fracos, no caso do XGBoost é utilizado árvore de decisão

In [ ]:
 xg_clf = xgb.XGBClassifier(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
X_train = X_train.reshape(len(X_train),3 * 96 * 96)
X_test = X_test.reshape(len(X_test),3 * 96 * 96)

In [ ]:
xg_clf.fit(X_train,y_train)

## Visualizado as métricas

In [ ]:
predictions  = xg_clf.predict(X_test)
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, predictions)
area_under_curve = auc(false_positive_rate, true_positive_rate)

plt.plot([0, 1], [0, 1], 'k--')
plt.plot(false_positive_rate, true_positive_rate, label='AUC = {:.3f}'.format(area_under_curve))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig('ROC_PLOT.png', bbox_inches='tight')
plt.show()



In [ ]:
print(accuracy_score(y_test, predictions))